In [1]:
from hijack import *

fovs 30


In [30]:

fov = 8
code = 1
fix_start,mov_start,last = starting[tuple([code,fov])]
with h5py.File(exseq.args.out_dir + 'code{}/{}_downsampled.h5'.format(exseq.args.ref_code,fov), 'r+') as f:
    print(exseq.args.out_dir + 'code{}/{}_downsampled.h5'.format(exseq.args.ref_code,fov))
    fix_vol = f['405'][fix_start:fix_start+last,:,:]

with h5py.File(exseq.args.out_dir + 'code{}/{}_downsampled.h5'.format(code,fov), 'r+') as f:
    mov_vol = f['405'][mov_start:mov_start+last,:,:]

/mp/nas3/ruihan/20221218_zebrafish//processed/code0/8_downsampled.h5


In [18]:
import SimpleITK as sitk
elastixImageFilter = sitk.ElastixImageFilter()


In [22]:


# cfg = load_cfg()
# align = alignBuild(cfg)
# align.buildSitkTile()

# fix_vol = nd2ToVol(exseq.args.mov_path.format(exseq.args.ref_code,'405',4), fov)
# fix_vol = fix_vol[starting['code{},fov{}'.format(exseq.args.ref_code,fov)]:,:,:]
# mov_vol = nd2ToVol(exseq.args.mov_path.format(code,'405',4), fov)
# mov_vol = mov_vol[starting['code{},fov{}'.format(code,fov)]:,:,:]

fix_vol_sitk = sitk.GetImageFromArray(fix_vol[:20,:30,:50])
fix_vol_sitk.SetSpacing([1.6,1.6,4])
elastixImageFilter.SetFixedImage(fix_vol_sitk)

mov_vol_sitk = sitk.GetImageFromArray(mov_vol[:20,:30,:50])
mov_vol_sitk.SetSpacing([1.6,1.6,4])
elastixImageFilter.SetMovingImage(mov_vol_sitk)

tform = sitk.ReadParameterFile('/mp/nas3/ruihan/20221218_zebrafish//processed/code1/tforms/8_downsampled.txt')
elastixImageFilter.SetParameterMap(tform)
parameterMap = elastixImageFilter.GetParameterMap()
sitk.PrintParameterMap(parameterMap)
# elastixImageFilter.Execute()
# sitk.WriteImage(elastixImageFilter.GetResultImage())


# tform = align.computeTransformMap(fix_vol, mov_vol)
# result = align.warpVolume(mov_vol, tform)
# with h5py.File(exseq.args.out_dir + 'code{}/{}_transformed.h5'.format(code,fov), 'w') as f:
#     f.create_dataset('405', result.shape, dtype=result.dtype, data = result)
#     align.writeTransformMap(exseq.args.out_dir + 'code{}/tforms/{}.txt'.format(code,fov), tform)

AttributeError: module 'SimpleITK' has no attribute 'readTransformMap'

In [23]:
parameterMap = sitk.GetDefaultParameterMap('translation')

In [29]:
sitk.PrintParameterMap(sitk.GetDefaultParameterMap("translation"))

In [ ]:
# import SimpleITK as sitk

# class ExmSitkTransform:
#     # input: nd2 and xlsx file
#     # output: on-demand volume at given downsample ratio
#     def __init__(self):
#         self.elastix = sitk.ElastixImageFilter()
#         self.transformix = sitk.TransformixImageFilter()
#         self.parameter_map = None
#         self.transform_map = None
#         self.transform_type = None
    
#     def setResolution(self, resolution):
#         # xyz-order
#         self.resolution = resolution

#     ## Volume
#     #### Setup
#     def setTransformType(self, transform_type, num_iteration = -1):
#         self.transform_type = transform_type
#         self.parameter_map = self.createParameterMap(transform_type, num_iteration)
#         self.elastix.SetParameterMap(self.parameter_map)
    
#     def setParameterMap(self, parameter_map):
#         self.parameter_map = parameter_map
#         self.elastix.SetParameterMap(parameter_map)

#     def setTransformMap(self, transform_map):
#         self.transform_map = transform_map

#     def getParameterMap(self):
#         return self.parameter_map

#     def getTransformMap(self):
#         return self.transform_map

#     def readTransformMap(self, filename, do_return=False):
#         self.transform_map = sitk.ReadParameterFile(filename)
#         if do_return:
#             return self.transform_map

#     def writeTransformMap(self, filename, transform_map=None):
#         if transform_map is None:
#             transform_map = self.transform_map
#         return sitk.WriteParameterFile(transform_map, filename)
    
#     def createParameterMap(self, transform_type = None, num_iteration = -1):
#         if transform_type is None:
#             transform_type = self.transform_type
#         if len(transform_type) == 1:
#             parameter_map = sitk.GetDefaultParameterMap(transform_type[0])
#             parameter_map['NumberOfSamplesForExactGradient'] = ['5000']
#             # important!!! o/w errors for not enough sampled points
#             #parameter_map['CheckNumberOfSamples'] = ['false']
#             if num_iteration > 0:
#                 parameter_map['MaximumNumberOfIterations'] = [str(num_iteration)]
#             else:
#                 parameter_map['MaximumNumberOfIterations'] = ['5000']
#             parameter_map['MaximumNumberOfSamplingAttempts'] = ['100']
#             parameter_map['FinalBSplineInterpolationOrder'] = ['1']            
#         else:
#             parameter_map = sitk.VectorOfParameterMap()
#             for trans in transform_type:
#                 parameter_map.append(self.createParameterMap(trans, num_iteration))
#         return parameter_map

#     #### Estimate and warp with transformation
#     def convertSitkImage(self, vol_np, res_np):
#         vol = sitk.GetImageFromArray(vol_np)
#         vol.SetSpacing(res_np)
#         # print(vol.GetPixelIDTypeAsString())
#         return vol
    
#     def computeTransformMap(self, vol_fix, vol_move, res_fix=None, res_move=None, mask_fix=None, mask_move=None, do_return=False):
#         # work with mask correctly
#         # https://github.com/SuperElastix/SimpleElastix/issues/198
#         # self.elastix.SetParameter("ImageSampler", "RandomSparseMask")
#         self.elastix.SetLogToConsole(False)
#         if res_fix is None:
#             res_fix = self.resolution
#         if res_move is None:
#             res_move = self.resolution
#         # 2. load volume
#         # print('vol-fix shape:', vol_fix.shape)
#         vol_fix = self.convertSitkImage(vol_fix, res_fix)
#         self.elastix.SetFixedImage(vol_fix)
#         if mask_fix is not None:
#             mask_fix = self.convertSitkImage(mask_fix, res_fix)
#             mask_fix.CopyInformation(vol_fix)
#             self.elastix.SetFixedMask(mask_fix)

#         # print('vol-move shape:', vol_move.shape)
#         vol_move = self.convertSitkImage(vol_move, res_move)
#         self.elastix.SetMovingImage(vol_move)
#         if mask_move is not None:
#             mask_move = self.convertSitkImage(mask_move, res_move)
#             mask_move.CopyInformation(vol_move)
#             self.elastix.SetMovingMask(mask_move)
            
#         # 3. compute transformation
#         self.elastix.Execute()

#         # 4. output transformation parameter
#         self.transform_map = self.elastix.GetTransformParameterMap()[0]
#         if do_return:
#             return self.transform_map
        
#     def warpVolume(self, vol_move, res_move=None, transform_map=None):
#         self.transformix.SetLogToConsole(False)
#         if res_move is None:
#             res_move = self.resolution
#         if transform_map is None:
#             transform_map = self.transform_map
#         self.transformix.SetTransformParameterMap(transform_map)
#         self.transformix.SetMovingImage(self.convertSitkImage(vol_move, res_move))
#         self.transformix.Execute()
#         out = sitk.GetArrayFromImage(self.transformix.GetResultImage())
#         return out

#     ## Point
#     #### Setup
#     def setPointTransformAffine(self, resolution = None, transform_map = None):
#         # https://github.com/SuperElastix/SimpleElastix/issues/91
#         # xyz order
#         if resolution is None:
#             resolution = self.resolution
#         if transform_map is None:
#             transform_map = self.transform_map
        
#         # um -> pix
#         param_val = np.array(transform_map["TransformParameters"]).astype(float)
#         param_cen = np.array([float(x) for x in transform_map["CenterOfRotationPoint"]])
#         self.A_A = param_val[:9].reshape([3,3])        
#         self.A_t = (param_val[9:] / resolution).reshape([-1,1])
#         self.A_cen = (param_cen / resolution).reshape([-1,1])        
        
#     def warpPoint(self, pts, mode = 'backward'):
#         # pts: 3xN matrix (xyz-order)
#         assert (np.array(pts.shape)==3).sum() > 0
#         if pts.shape[0] != 3:
#             pts = pts.T

#         # A = computeTransformMap(vol_fixed, vol_moving)
#         # for pts from vol_moving, need the "inverse" mode to warp it to vol_fixed

#         if mode == 'backward':
#             pts_out = np.linalg.solve(self.A_A, pts - self.A_cen - self.A_t) + self.A_cen
#         elif mode == 'forward':
#             pts_out = np.matmul(self.A_A, pts - self.A_cen) + self.A_t + self.A_cen
#         return pts_out

In [ ]:
# # 1. set transformation parameters
# sitkWrapper = ExmSitkTransform()
# # zyx -> xyz order
# sitkWrapper.setResolution(raw_data.getResolution(ratio_low)[::-1])
# sitkWrapper.setTransformType(['affine'], num_iteration=1000)

# sn_param = Do + 'affine_low-res.txt'
# if not os.path.exists(sn_param):
#     sitkWrapper.computeTransformMap(stitched_vol, raw_data_vol,\
#                                     mask_fix = (stitched_vol>0).astype(np.uint8),\
#                                     mask_move = (raw_data_vol>0).astype(np.uint8))
#     sitkWrapper.writeTransformMap(sn_param)
# else:
#     sitkWrapper.readTransformMap(sn_param)

In [ ]:
# transform_local = sitkWrapper.readTransformMap(sn_param, True)
# transform_local["Size"] = tuple([str(x) for x in output_size[::-1]])
# trans0_um = np.array([float(x) for x in transform_local["TransformParameters"]])
# cen_global_um = np.array([float(x) for x in transform_local['CenterOfRotationPoint']])


# # local refinement
# # https://webcache.googleusercontent.com/search?q=cache:5TupR2uef-wJ:https://elastix.lumc.nl/doxygen/classelastix_1_1AdaptiveStochasticGradientDescent.html+&cd=1&hl=en&ct=clnk&gl=us
# sitkWrapper.setResolution(raw_data.getResolution(ratio_warp)[::-1])
# sitkWrapper.setTransformType([transform_type], 2000)
# sitkWrapper.parameter_map['NumberOfResolutions'] = ['1']
# #sitkWrapper.parameter_map['ImageSampler'] = ['RandomSparseMask']
# #sitkWrapper.parameter_map['Optimizer'] = ['AdaGrad']
# #sitkWrapper.parameter_map['Metric'] = ['NormalizedMutualInformation']
# #sitkWrapper.parameter_map['NumberOfSpatialSamples'] = ['512']
# #sitkWrapper.parameter_map['NumberOfSamplesForExactGradient'] = ['1000']
# #### KEY: turn off the check !!!
# # low-res bug: not enough points
# # sitkWrapper.parameter_map['CheckNumberOfSamples'] = ['false']
# #sitkWrapper.parameter_map['MaximumNumberOfSamplingAttempts'] = ['10000']
# sitkWrapper.setParameterMap(sitkWrapper.parameter_map)

#  trans_um = trans0_um.copy()
#         trans_um[9:] = shift_local_um
#         transform_local["TransformParameters"] = tuple([str(x) for x in trans_um])        
#         cen_tile_um = [str(x) for x in cen_global_um - (pts_um + shift_local_um)]
#         transform_local['CenterOfRotationPoint'] = tuple(cen_tile_um)
#         raw_data_tile_w1 = sitkWrapper.warpVolume(raw_data_tile, transform_map=transform_local)
#        sitkWrapper.computeTransformMap